In [65]:
import pandas as pd
import requests
import numpy as np
import re
from bs4 import BeautifulSoup as bs

In [66]:
data = pd.read_excel("Aktualni-data-STK-na-web.xlsx", header=1)
data = data.rename(columns={"Unnamed: 1": "Rozsah oprávnění"})

data['STK č.'] = data['STK č.'].astype(str).str.replace('.','')

In [67]:
len(data.Město.unique())

257

In [68]:
display(data)

#ORP - obec s rozšířenou působností 

,STK č.,Rozsah oprávnění,PSČ,Město,Ulice,Provozovatel STK,Telefon,E-mail,ORP,Okres,Kraj
0,310,"OA, NA, TRA, ZS, ADR",149 00,Praha 4,Türkova 1001,DEKRA CZ a.s.,"267 913 838, 267 288 245",stk@dekra.cz,Praha,Praha,Hlavní město Praha
1,3102,"OA, ZS",190 00,Praha 9,Spojovací 41,DEKRA CZ a.s.,284829315,stk-spojovaci@dekra.cz,Praha,Praha,Hlavní město Praha
2,3103,"NA, TRA",108 56,Praha 10,U vozovny 6,Dopravní podnik hl.m. Prahy a.s.,272704945,stk@dpp.cz,Praha,Praha,Hlavní město Praha
3,3104,"OA, ZS",150 00,Praha 5,Plzeňská 215b,STK MOTOL S.R.O.,257210963,stk@stkmotol.cz,Praha,Praha,Hlavní město Praha
4,3105,"OA, ZS",100 31,Praha 10,Průběžná 76,"STK Praha, s.r.o.",274774867,stk@aboservis.cz,Praha,Praha,Hlavní město Praha
...,...,...,...,...,...,...,...,...,...,...,...
367,681,TRA,783 11,Štěpánov,Horní 11,ECCO-TRAC s.r.o.,585 386 558,tiefenbach@centrum.cz,Olomouc,Olomouc,Olomoucký kraj
368,6811,TRA,750 00,Přerov,9.května-areál STS,ESTEKO Přerov v.o.s.,581204898,esteko@seznam.cz,Přerov,Přerov,Olomoucký kraj
369,6813,TRA,779 00,Olomouc,Pode Mlýnem 763/11,Pro - Truck servis Olomouc s.r.o.,731140142,jarda.kucera@centrum.cz,Olomouc,Olomouc,Olomoucký kraj
370,6814,TRA,753 01,Hranice,Tovární 605,ADR Logistik s.r.o.,777738599,davidrichter@seznam.cz,Hranice,Přerov,Olomoucký kraj


In [69]:
data['Rozsah oprávnění'].unique()

# OA = Osobní automobil
# NA = Nákladní automobil
# TRA = Traktor
# ZS = Zkušební stanice
# ADR = Přeprava nebezpečného nákladu

array(['OA, NA, TRA, ZS, ADR', 'OA, ZS', 'NA, TRA', 'NA, TRA, ZS',
       'OA, NA, TRA, ZS', 'OA', 'OA, TRA', 'OA, TRA, ZS', 'OA, NA, TRA',
       'NA, TRA, ZS, ADR', 'OA, NA, TRA, ADR, ZS', 'TRA'], dtype=object)

In [91]:
cenaSTK      = []
cenaEvKont   = []
cenaEmiseB   = []
cenaEmiseN   = []
mesta        = []
addr         = []
nazev        = []
href         = []
provoznidoba = []

In [71]:
#scraping prices from mbenzin.cz (by city/adress/etc.)

for mesto in data['Město'].unique():
    #zkousela jsem tam davat ulice/ORP/Okres/Kraj misto Mesta, ale pocet nalezenych stanic byl vzdy stejny - 289 
    url      = 'https://www.mbenzin.cz/STK/{0}'.format(mesto) 
    response = requests.get(url)
    soup     = bs(response.text, "html.parser")
    
    for i in soup.findAll("div", {"class": "row no-gutters px-3 py-2 text-lh-20"}):
        total = []
        nazev.append(i.find("span", {"class": "font-s-11r"}).get_text())
        
        for t in i.findAll("div", {"class": "col text-center p-1"}):
            cena = t.find("div", {"class": "align-middle font-s-12r text-primary"}).get_text() if t.find("div", {"class": "align-middle font-s-12r text-primary"}) else "-" 
            total.append(cena)
        
        cenaSTK.append(total[0])
        cenaEvKont.append(total[1])
        cenaEmiseB.append(total[2])
        cenaEmiseN.append(total[3])
        
        href.append(i.find("a")['href'])
        
        mesta.append(i.find("span", {"itemprop": "addressLocality"}).get_text())
        addr.append(i.find("span", {"itemprop": "streetAddress"}).get_text())

In [122]:
df = pd.DataFrame({'nazev' : nazev, 'cenaSTK' : cenaSTK, 'cenaEvKont' : cenaEvKont, 'cenaEmiseB' : cenaEmiseB, 'cenaEmiseN' : cenaEmiseN, 'mesta' : mesta, 'addr' : addr, 'href' : href})
df = df.drop_duplicates()

In [92]:
#scraping working hours from mbenzin.cz

for index, row in df.iterrows():
    url      = 'https://www.mbenzin.cz/STK/{0}'.format(row['href'])
    response = requests.get(url)
    soup     = bs(response.text, "html.parser")
    
    if soup.find("div", {"class": "col pl-2"}):
        col = soup.find("div", {"class": "col pl-2"})
        if col.find("span", {"id": "ContentPlaceHolder1_lDescription"}):
            provoznidoba.append(col.find("span", {"id": "ContentPlaceHolder1_lDescription"}).get_text())
        else:
            provoznidoba.append("-")
    else:
        provoznidoba.append("-")

In [123]:
#has to be parsed

display(provoznidoba[1:10])

['Po-Čt: 7.00-17.30 \r\nPá: 7.00-15.30',
 'Po-Čt: 7-18, Pá: 7-15',
 'Provozní doba\r\nPo-Čt: 7:00-18:00\r\nPá: 7:00-14:00',
 'Nákladní auta > Dobronická 635\nProvozní doba\r\nPo-Čt: 7:00-16:30\r\nPá: 7:00-12:00',
 'Po,Út,Čt: 7:00-15:30\r\nSt: 7:00-18:00\r\nPá: 7:00-12:00',
 'Po, St: 7-17\r\nÚt, Čt: 7-15\r\nPá: 7:30-13',
 'Po-Pá: 7:30-18:00\r\nobjekt AutoFREIBERG',
 'Provozní doba\r\nPo-Čt: 7:00-19:00\r\nPá: 7:00-17:00',
 'Provozní doba\r\nPo-Čt: 7:30 - 18:00\r\nPá: 7:30 - 17:00']

In [ ]:
#naive parsing 

for index, row in df.iterrows():
    tmp = row['pracovniDoba'].replace(' ', '')
    if row['pracovniDoba'] == '-':
            df.loc[index, 'Po'] = '-'
            df.loc[index, 'Ut'] = '-'
            df.loc[index, 'St'] = '-'
            df.loc[index, 'Ct'] = '-'
            df.loc[index, 'Pa'] = '-'
            df.loc[index, 'So'] = '-'
            df.loc[index, 'Ne'] = '-'
    if re.match(r'.*Po-Pá:.*', tmp):
        tmp = tmp[tmp.index('Po-Pá:') + len('Po-Pá:'):]
        if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
            df.loc[index, 'Po'] = tmp[:10]
            df.loc[index, 'Ut'] = tmp[:10]
            df.loc[index, 'St'] = tmp[:10]
            df.loc[index, 'Ct'] = tmp[:10]
            df.loc[index, 'Pa'] = tmp[:10]
            
            if re.match(r'.*So:.*', tmp[10:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:10]
            
            if re.match(r'.*Ne:.*', tmp[10:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:10]
                
        if re.match(r'\d-\d\d.*', tmp):
            df.loc[index, 'Po'] = tmp[:4]
            df.loc[index, 'Ut'] = tmp[:4]
            df.loc[index, 'St'] = tmp[:4]
            df.loc[index, 'Ct'] = tmp[:4]
            df.loc[index, 'Pa'] = tmp[:4]
            
            if re.match(r'.*So:.*', tmp[4:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:4]
                
            if re.match(r'.*Ne:.*', tmp[4:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:4]
                
    tmp = row['pracovniDoba'].replace(' ', '')      
    if re.match(r'.*Po-Čt:.*', tmp):
        tmp = tmp[tmp.index('Po-Čt:') + len('Po-Čt:'):]
        if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
            df.loc[index, 'Po'] = tmp[:10]
            df.loc[index, 'Ut'] = tmp[:10]
            df.loc[index, 'St'] = tmp[:10]
            df.loc[index, 'Ct'] = tmp[:10]
            
            if re.match(r'.*Pá:.*', tmp[10:]):
                tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
                df.loc[index, 'Pa'] = tmp[:10]
            
            if re.match(r'.*So:.*', tmp[10:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:10]
                
            if re.match(r'.*Ne:.*', tmp[10:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:10]
            
        if re.match(r'\d-\d\d.*', tmp):
            df.loc[index, 'Po'] = tmp[:4]
            df.loc[index, 'Ut'] = tmp[:4]
            df.loc[index, 'St'] = tmp[:4]
            df.loc[index, 'Ct'] = tmp[:4]
            
            if re.match(r'.*Pá:.*', tmp[4:]):
                tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
                df.loc[index, 'Pa'] = tmp[:4]
            
            if re.match(r'.*So:.*', tmp[4:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:4]
                
            if re.match(r'.*Ne:.*', tmp[4:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:4]
                
    tmp = row['pracovniDoba'].replace(' ', '')               
    if re.match(r'.*Po,St:.*', tmp):
        tmp = tmp[tmp.index('Po,St:') + len('Po,St:'):]
        if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
            df.loc[index, 'Po'] = tmp[:10]
            df.loc[index, 'St'] = tmp[:10]
            
            if re.match(r'.*Pá:.*', tmp[10:]):
                tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
                df.loc[index, 'Pa'] = tmp[:10]
            
            if re.match(r'.*So:.*', tmp[10:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:10]
                
            if re.match(r'.*Ne:.*', tmp[10:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:10]
            
        if re.match(r'\d-\d\d.*', tmp):
            df.loc[index, 'Po'] = tmp[:4]
            df.loc[index, 'St'] = tmp[:4]
            
            if re.match(r'.*Pá:.*', tmp[4:]):
                tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
                df.loc[index, 'Pa'] = tmp[:4]
            
            if re.match(r'.*So:.*', tmp[4:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:4]
                
            if re.match(r'.*Ne:.*', tmp[4:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:4]
                
    tmp = row['pracovniDoba'].replace(' ', '')                
    if re.match(r'.*Út,Čt:.*', tmp):
        tmp = tmp[tmp.index('Út,Čt:') + len('Út,Čt:'):]
        if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
            df.loc[index, 'Ut'] = tmp[:10]
            df.loc[index, 'Ct'] = tmp[:10]
            
            if re.match(r'.*Pá:.*', tmp[10:]):
                tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
                df.loc[index, 'Pa'] = tmp[:10]
            
            if re.match(r'.*So:.*', tmp[10:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:10]
                
            if re.match(r'.*Ne:.*', tmp[10:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:10]
            
        if re.match(r'\d-\d\d.*', tmp):
            df.loc[index, 'Ut'] = tmp[:4]
            df.loc[index, 'Ct'] = tmp[:4]
            
            if re.match(r'.*Pá:.*', tmp[4:]):
                tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
                df.loc[index, 'Pa'] = tmp[:4]
            
            if re.match(r'.*So:.*', tmp[4:]):
                tmp = tmp[tmp.index('So:') + len('So:'):]
                df.loc[index, 'So'] = tmp[:4]
                
            if re.match(r'.*Ne:.*', tmp[4:]):
                tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
                df.loc[index, 'Ne'] = tmp[:4]
                
    tmp = row['pracovniDoba'].replace(' ', '')                
    if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
        if re.match(r'.*Po:.*', tmp[10:]):
            tmp = tmp[tmp.index('Po:') + len('Po:'):]
            df.loc[index, 'Po'] = tmp[:10]

        if re.match(r'.*Út:.*', tmp[10:]):
            tmp = tmp[tmp.index('Út:') + len('Út:'):]
            df.loc[index, 'Út'] = tmp[:10]
            
        if re.match(r'.*St:.*', tmp[10:]):
            tmp = tmp[tmp.index('St:') + len('St:'):]
            df.loc[index, 'St'] = tmp[:10]
        
        if re.match(r'.*Čt:.*', tmp[10:]):
            tmp = tmp[tmp.index('Čt:') + len('Čt:'):]
            df.loc[index, 'Čt'] = tmp[:10]
            
        if re.match(r'.*Pá:.*', tmp[10:]):
            tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
            df.loc[index, 'Pa'] = tmp[:10]

        if re.match(r'.*So:.*', tmp[10:]):
            tmp = tmp[tmp.index('So:') + len('So:'):]
            df.loc[index, 'So'] = tmp[:10]

        if re.match(r'.*Ne:.*', tmp[10:]):
            tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
            df.loc[index, 'Ne'] = tmp[:10]
            
    tmp = row['pracovniDoba'].replace(' ', '')
    if re.match(r'\d-\d\d.*', tmp):
        if re.match(r'.*Po:.*', tmp[4:]):
            tmp = tmp[tmp.index('Po:') + len('Po:'):]
            df.loc[index, 'Po'] = tmp[:4]

        if re.match(r'.*Út:.*', tmp[4:]):
            tmp = tmp[tmp.index('Út:') + len('Út:'):]
            df.loc[index, 'Ut'] = tmp[:4]
            
        if re.match(r'.*St:.*', tmp[4:]):
            tmp = tmp[tmp.index('St:') + len('St:'):]
            df.loc[index, 'St'] = tmp[:4]
        
        if re.match(r'.*Čt:.*', tmp[4:]):
            tmp = tmp[tmp.index('Čt:') + len('Čt:'):]
            df.loc[index, 'Ct'] = tmp[:4]
        if re.match(r'.*Pá:.*', tmp[4:]):
            tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
            df.loc[index, 'Pa'] = tmp[:4]

        if re.match(r'.*So:.*', tmp[4:]):
            tmp = tmp[tmp.index('So:') + len('So:'):]
            df.loc[index, 'So'] = tmp[:4]

        if re.match(r'.*Ne:.*', tmp[4:]):
            tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
            df.loc[index, 'Ne'] = tmp[:4]

In [124]:
df['pracovniDoba'] = provoznidoba

In [125]:
df = df.drop_duplicates(subset=['nazev', 'addr'])
display(df.sample(10))
df.info()

,nazev,cenaSTK,cenaEvKont,cenaEmiseB,cenaEmiseN,mesta,addr,href,pracovniDoba
3605,STK Jablonné s.r.o.,950 Kč,300 Kč,500 Kč,850 Kč,Jablonné nad Orlicí,U Koupaliště 180,Jablonne-nad-Orlici/STK-Jablonne-sro-U-Koupali...,"Provozní doba\r\nPo,St: 7:00-17:00\r\nÚt,Čt: 7..."
5447,STK Mariánské Hory,1200 Kč,500 Kč,700 Kč,800 Kč,Mariánské Hory,Oběžná 2086/13,Marianske-Hory/STK-Marianske-Hory-Obezna-20861...,Tel: 724 177 911\nProvozní doba\r\nNa objednáv...
4554,STK Brno Černovice,1300 Kč,500 Kč,1200 Kč,1200 Kč,Brno - Černovice,Olomoucká 1283/156c,Brno---Cernovice/STK-Brno-Cernovice-Olomoucka-...,+420 777 662 005\r\nProvozní doba\r\nPo-Pá 7:3...
4503,STK Dolní Vilémovice,900 Kč,250 Kč,550 Kč,800 Kč,Lipník,Dolní Vilémovice 148,Lipnik/STK-Dolni-Vilemovice-Dolni-Vilemovice-1...,Po-Pá: 6-17
1276,"STK Sovětice CZ, s.r.o.",900 Kč,400 Kč,650 Kč,750 Kč,Nechanice,Sovětice 19,Nechanice/STK-Sovetice-CZ-sro-Sovetice-19/19792,-
2133,"TÜV NORD Czech, s.r.o.",1200 Kč,450 Kč,700 Kč,850 Kč,Karlovy Vary - Bohatice,Hroznětínská 4,Karlovy-Vary---Bohatice/TUV-NORD-Czech-sro-Hro...,Provozní doba\r\nPo-Čt: 6:30 - 18:00Pá: 6:30-1...
5489,JV Foods s.r.o,750 Kč,350 Kč,600 Kč,650 Kč,Bruntál,Zahradní 44,Bruntal/JV-Foods-sro-Zahradni-44/1365,"Tel: 606 703 885\r\nPo, St: 7.00 - 17.00\r\nÚt..."
4767,STK KOPR s.r.o.,1150 Kč,490 Kč,1000 Kč,1000 Kč,Kopřivnice,Štefánikova 221/20H,Koprivnice/STK-KOPR-sro-Stefanikova-22120H/1375,+420 603 788 782\r\nPo-Čt: 7:00-17:00 ...
4549,STK Brno Líšeň,990 Kč,500 Kč,950 Kč,950 Kč,Brno,Trnkova 2785/144a,Brno/STK-Brno-Lisen-Trnkova-2785144a/19603,Provozní doba\r\nPo-Pá: 7:00-18:00
1584,Opravny Telč,800 Kč,300 Kč,500 Kč,700 Kč,Telč,Radkovská 230,Telc/Opravny-Telc-Radkovska-230/1353,"Provozní doba\r\nPo,St: 7-17\r\nÚt,Čt: 7-15:30..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 0 to 6167
Data columns (total 9 columns):
nazev           288 non-null object
cenaSTK         288 non-null object
cenaEvKont      288 non-null object
cenaEmiseB      288 non-null object
cenaEmiseN      288 non-null object
mesta           288 non-null object
addr            288 non-null object
href            288 non-null object
pracovniDoba    288 non-null object
dtypes: object(9)
memory usage: 22.5+ KB


In [126]:
#removing Kč
df['cenaSTK']    = df['cenaSTK'].astype(str).str.replace(' Kč','')
df['cenaEvKont'] = df['cenaEvKont'].astype(str).str.replace(' Kč','')
df['cenaEmiseB'] = df['cenaEmiseB'].astype(str).str.replace(' Kč','')
df['cenaEmiseN'] = df['cenaEmiseN'].astype(str).str.replace(' Kč','')

df['cenaSTK']    = df['cenaSTK'].astype(str).str.replace('-','0')
df['cenaEvKont'] = df['cenaEvKont'].astype(str).str.replace('-','0')
df['cenaEmiseB'] = df['cenaEmiseB'].astype(str).str.replace('-','0')
df['cenaEmiseN'] = df['cenaEmiseN'].astype(str).str.replace('-','0')

In [127]:
df['pracovniDoba'] = df['pracovniDoba'].astype(str).str.replace('#','')
df['pracovniDoba'] = df['pracovniDoba'].astype(str).str.replace('\n',' ')
df['pracovniDoba'] = df['pracovniDoba'].astype(str).str.replace('\r',' ')
df = df.fillna(0)

In [128]:
data['cenaSTK'] = np.nan
data['cenaEvKont'] = np.nan
data['cenaEmiseB'] = np.nan
data['cenaEmiseN'] = np.nan
data['pracovniDoba'] = np.nan

In [129]:
#address is unique - datasets can be joined by address
len(data.Ulice.unique())

for index, row in df.iterrows():
    data.loc[data['Ulice'] == row['addr'], 'cenaSTK'] = row['cenaSTK']
    data.loc[data['Ulice'] == row['addr'], 'cenaEvKont'] = row['cenaEvKont']
    data.loc[data['Ulice'] == row['addr'], 'cenaEmiseB'] = row['cenaEmiseB']
    data.loc[data['Ulice'] == row['addr'], 'cenaEmiseN'] = row['cenaEmiseN']
    data.loc[data['Ulice'] == row['addr'], 'pracovniDoba'] = row['pracovniDoba']
data = data.fillna(0)

In [130]:
#import to db

import psycopg2

data.to_csv('no_header_stk.csv', header=False, index=False, sep='#')
data.to_csv('with_header_stk.csv', header=True, index=False, sep=',')
conn = psycopg2.connect("host='193.85.191.172' port='5432' dbname='odl_stk_data' user='odl_stk_data' password='Aj324oi43j24'")
cur = conn.cursor()
f = open(r'no_header_stk.csv', 'r', encoding="utf8")
cur.copy_from(f, 'seznam_stk', sep='#')
f.close()
conn.commit()
conn.close()

In [131]:
data

,STK č.,Rozsah oprávnění,PSČ,Město,Ulice,Provozovatel STK,Telefon,E-mail,ORP,Okres,Kraj,cenaSTK,cenaEvKont,cenaEmiseB,cenaEmiseN,pracovniDoba
0,310,"OA, NA, TRA, ZS, ADR",149 00,Praha 4,Türkova 1001,DEKRA CZ a.s.,"267 913 838, 267 288 245",stk@dekra.cz,Praha,Praha,Hlavní město Praha,1040,390,740,1140,Provozní doba Po-Čt: 7:00-18:00 Pá: 7:00-14:00
1,3102,"OA, ZS",190 00,Praha 9,Spojovací 41,DEKRA CZ a.s.,284829315,stk-spojovaci@dekra.cz,Praha,Praha,Hlavní město Praha,1050,450,650,850,"Po-Čt: 7-18, Pá: 7-11:30 Provozovatel Dekra C..."
2,3103,"NA, TRA",108 56,Praha 10,U vozovny 6,Dopravní podnik hl.m. Prahy a.s.,272704945,stk@dpp.cz,Praha,Praha,Hlavní město Praha,0,0,0,0,0
3,3104,"OA, ZS",150 00,Praha 5,Plzeňská 215b,STK MOTOL S.R.O.,257210963,stk@stkmotol.cz,Praha,Praha,Hlavní město Praha,940,400,690,940,"Po, St: 7-17 Út, Čt: 7-15 Pá: 7:30-13"
4,3105,"OA, ZS",100 31,Praha 10,Průběžná 76,"STK Praha, s.r.o.",274774867,stk@aboservis.cz,Praha,Praha,Hlavní město Praha,880,390,550,750,"Po,Út,Čt: 7:00-15:30 St: 7:00-18:00 Pá: 7:00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,681,TRA,783 11,Štěpánov,Horní 11,ECCO-TRAC s.r.o.,585 386 558,tiefenbach@centrum.cz,Olomouc,Olomouc,Olomoucký kraj,0,0,0,0,0
368,6811,TRA,750 00,Přerov,9.května-areál STS,ESTEKO Přerov v.o.s.,581204898,esteko@seznam.cz,Přerov,Přerov,Olomoucký kraj,0,0,0,0,0
369,6813,TRA,779 00,Olomouc,Pode Mlýnem 763/11,Pro - Truck servis Olomouc s.r.o.,731140142,jarda.kucera@centrum.cz,Olomouc,Olomouc,Olomoucký kraj,0,0,0,0,0
370,6814,TRA,753 01,Hranice,Tovární 605,ADR Logistik s.r.o.,777738599,davidrichter@seznam.cz,Hranice,Přerov,Olomoucký kraj,0,0,0,0,0
